In [1]:
# !pip install -U torchdata
# !pip install -U spacy
# !python -m spacy download en_core_web_lg
# !python -m spacy download de_core_news_lg
# !pip install portalocker
! pip install accelerate -U
! pip install evaluate
! pip install sentence-transformers
! pip install SentencePiece
! pip install bert_score
! pip install --upgrade nltk

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 163.3/163.3 kB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.1/61.1 kB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 31.8 MB/s eta 0:00:00
  Attempting uninstall: nltk
    Found existing installation: nltk 3.2.4
    Uninstalling nltk-3.2.4:
      Successfully uninstalled nltk-3.2.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
preprocessing 0.1.13 requires nltk==3.2.4, but you have nltk 3.8.1 which is incompatible.


In [2]:
import numpy as np
import pandas as pd
import json
import torch

import logging
import evaluate
from evaluate import load
# from torchtext.data.utils import get_tokenizer
# from torchtext.vocab import build_vocab_from_iterator
# from typing import Iterable, List
# from torch import Tensor
# import torch.nn as nn
# from torch.nn import Transformer
# import math
# from torch.nn.utils.rnn import pad_sequence
# from torch.utils.data import DataLoader
# from tqdm import tqdm
# from timeit import default_timer as timer
import datasets
from datasets import load_dataset

from torch.utils.data import Dataset, DataLoader
from transformers import AutoTokenizer
from transformers import AutoModelForSeq2SeqLM, Seq2SeqTrainingArguments, Seq2SeqTrainer
from transformers import T5Tokenizer,T5ForConditionalGeneration

import nltk
from nltk.translate.bleu_score import corpus_bleu

2024-03-28 19:33:27.639541: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-03-28 19:33:27.639674: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-03-28 19:33:27.776364: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [3]:
device=""
if torch.cuda.is_available():
    device = torch.device("cuda")
    logging.info(f'Using GPU: {torch.cuda.get_device_name()}')
    print(f'Using GPU: {torch.cuda.get_device_name()}')
else:
    device = torch.device("cpu")
    logging.info('Using CPU')
    print('Using CPU')
    
print("proof")

Using GPU: Tesla P100-PCIE-16GB
proof


In [4]:
SRC_LANGUAGE = 'de'
TGT_LANGUAGE = 'en'
PATH='./data/Task2/'

In [5]:
T5_model = "t5-small"
tokenizer = AutoTokenizer.from_pretrained(T5_model)

tokenizer_config.json:   0%|          | 0.00/2.32k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

In [6]:
# token_transform[SRC_LANGUAGE] = get_tokenizer('spacy', language='de_core_news_lg')
# token_transform[TGT_LANGUAGE] = get_tokenizer('spacy', language='en_core_web_lg')

In [7]:
# def yield_tokens(data_iter: Iterable, language: str) -> List[str]:
#     language_index = {SRC_LANGUAGE: 0, TGT_LANGUAGE: 1}

#     for data_sample in data_iter:
#         yield token_transform[language](data_sample[language_index[language]])

In [8]:
train_data=datasets.load_dataset('wmt16',"de-en", split="train[:50000]")
val_data=datasets.load_dataset('wmt16',"de-en", split="validation")
test_data=datasets.load_dataset('wmt16',"de-en", split="test")
# train_data=load_dataset("wmt16","de-en", split="train[500:1000]")
# val_data=load_dataset("wmt16","de-en", split="validation[:50]")
# test_data=load_dataset("wmt16","de-en", split="test[:50]")

Extracting data files:   0%|          | 0/5 [00:00<?, ?it/s]

Extracting data files: 0it [00:00, ?it/s]

Generating train split:   0%|          | 0/4548885 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/2169 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/2999 [00:00<?, ? examples/s]

Dataset wmt16 downloaded and prepared to /root/.cache/huggingface/datasets/wmt16/de-en/1.0.0/9e0038fe4cc117bd474d2774032cc133e355146ed0a47021b2040ca9db4645c0. Subsequent calls will reuse this data.


In [9]:

def preprocess_data(data):
    en_data = []
    de_data = []
    data = data["translation"]
    prefix = "translate German to English: "
    for i in range(len(data)):
        en_data.append(data[i]["en"])
        de_data.append(prefix+data[i]["de"])

    model_inputs = tokenizer(de_data, text_target=en_data, max_length=128, truncation=True)
    return model_inputs


In [10]:
tokenized_data = train_data.map(preprocess_data, batched=True)
val_tokenized_data = val_data.map(preprocess_data, batched=True)
test_tokenized_data = test_data.map(preprocess_data, batched=True)

  0%|          | 0/50 [00:00<?, ?ba/s]

  0%|          | 0/3 [00:00<?, ?ba/s]

  0%|          | 0/3 [00:00<?, ?ba/s]

In [11]:
from transformers import DataCollatorForSeq2Seq

data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model=T5_model)

In [12]:
bertscore = load("bertscore")
meteor = evaluate.load("meteor")
bleu = evaluate.load("bleu")

[nltk_data] Downloading package wordnet to /usr/share/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to /usr/share/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /usr/share/nltk_data...


In [13]:

def postprocess_text(preds, labels):
    preds = [pred.strip() for pred in preds]
    labels = [[label.strip()] for label in labels]

    return preds, labels


def compute_metrics(eval_preds):
    preds, labels = eval_preds
    if isinstance(preds, tuple):
        preds = preds[0]
    decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)

    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    pred, ref = postprocess_text(decoded_preds, decoded_labels)

    # cal bleu, meteor and bert
    bleu_score = bleu.compute(predictions=pred, references=ref)
#     bleu1 = corpus_bleu(ref,pred,weights=(1,0,0,0))
#     bleu2 = corpus_bleu(ref,pred,weights=(0.5,0.5,0))
#     bleu3 = corpus_bleu(ref,pred,weights=(0.33,0.33,0.33,0)
#     bleu4 = corpus_bleu(ref,pred,weights=(0.25,0.25,0.25,0.25))

    print(bleu_score)
    meteor_score = meteor.compute(predictions=pred, references=ref)
    bertscore_score = bertscore.compute(predictions=pred, references=ref, lang="en")

    #  dict
    return {"blue":bleu_score["bleu"],
            "precisions_1":bleu_score["precisions"][0],
            "precisions_2":bleu_score["precisions"][1],
            "precisions_3":bleu_score["precisions"][2],
            "precisions_4":bleu_score["precisions"][3],
            "bp":bleu_score["brevity_penalty"],
            "meteor": meteor_score["meteor"], 
            "bertscore_f1": np.average(bertscore_score["f1"]), 
            "bertscore_p": np.average(bertscore_score["precision"]), 
            "bertscore_r": np.average(bertscore_score["recall"])}


In [14]:
model = AutoModelForSeq2SeqLM.from_pretrained(T5_model)
model.to(device)

config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/242M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

T5ForConditionalGeneration(
  (shared): Embedding(32128, 512)
  (encoder): T5Stack(
    (embed_tokens): Embedding(32128, 512)
    (block): ModuleList(
      (0): T5Block(
        (layer): ModuleList(
          (0): T5LayerSelfAttention(
            (SelfAttention): T5Attention(
              (q): Linear(in_features=512, out_features=512, bias=False)
              (k): Linear(in_features=512, out_features=512, bias=False)
              (v): Linear(in_features=512, out_features=512, bias=False)
              (o): Linear(in_features=512, out_features=512, bias=False)
              (relative_attention_bias): Embedding(32, 8)
            )
            (layer_norm): T5LayerNorm()
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (1): T5LayerFF(
            (DenseReluDense): T5DenseActDense(
              (wi): Linear(in_features=512, out_features=2048, bias=False)
              (wo): Linear(in_features=2048, out_features=512, bias=False)
              (dropout): Drop

In [15]:
# ! rm -rf T53

In [16]:
import wandb
api_key = "9963cf6219e451d47251ea34645181ada1b2526b"
wandb.login(key=api_key)

wandb: W&B API key is configured. Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

In [17]:
# ! rm -rf "/kaggle/wandb"

In [18]:
training_args = Seq2SeqTrainingArguments(
    output_dir="./T5",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=25,
    per_device_eval_batch_size=25,
    weight_decay=0.01,
    save_total_limit=7,
    num_train_epochs=5,
    predict_with_generate=True,
    save_strategy="epoch",
    fp16=True,
#     report_to="wandb",
#     logging_dir='./lolololol'

    # push_to_hub=True
    # load_best_model_at_end=True,
    # metric_for_best_model="bleu"
)

trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_data,
    eval_dataset=val_tokenized_data,
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

trainer.train()

/opt/conda/lib/python3.10/site-packages/accelerate/accelerator.py:432: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(
wandb: Currently logged in as: raghav21274 (ragha). Use `wandb login --relogin` to force relogin
wandb: wandb version 0.16.5 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade
wandb: Tracking run with wandb version 0.16.4
wandb: Run data is saved locally in /kaggle/working/wandb/run-20240328_193840-lo5enhkl
wandb: Run `wandb offline` to turn off syncing.
wandb: Syncing run rare-gorge-5
wandb: ⭐️ View project at https://wandb.ai/ragha/huggingface
wandb: 🚀 View run at h

Epoch,Training Loss,Validation Loss,Blue,Precisions 1,Precisions 2,Precisions 3,Precisions 4,Bp,Meteor,Bertscore F1,Bertscore P,Bertscore R
1,2.409000,2.144973,0.079697,0.486918,0.196666,0.095874,0.051560,0.540309,0.310170,0.886250,0.890639,0.882312
2,2.305200,2.102104,0.087585,0.499462,0.212924,0.107957,0.059137,0.542585,0.326486,0.889562,0.894828,0.884728
3,2.247300,2.081212,0.087649,0.501826,0.214071,0.108585,0.059273,0.540521,0.327613,0.890285,0.895646,0.885357
4,2.223600,2.071012,0.089033,0.503589,0.215802,0.110154,0.060293,0.543192,0.329444,0.890567,0.896029,0.885543
5,2.211400,2.067840,0.089315,0.503341,0.215970,0.110242,0.060398,0.544526,0.329933,0.890679,0.896241,0.885550


/opt/conda/lib/python3.10/site-packages/transformers/generation/utils.py:1178: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


{'bleu': 0.07969687955403569, 'precisions': [0.48691809894927285, 0.19666578858239567, 0.09587413446961937, 0.05155983098085049], 'brevity_penalty': 0.5403086223746585, 'length_ratio': 0.6189594280299767, 'translation_length': 28742, 'reference_length': 46436}


tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

...

/opt/conda/lib/python3.10/site-packages/transformers/generation/utils.py:1178: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


{'bleu': 0.08903336765182861, 'precisions': [0.5035891389534279, 0.21580170991450429, 0.11015427312056159, 0.06029273532966295], 'brevity_penalty': 0.5431920530262271, 'length_ratio': 0.6210052545438883, 'translation_length': 28837, 'reference_length': 46436}


/opt/conda/lib/python3.10/site-packages/transformers/generation/utils.py:1178: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


{'bleu': 0.08931505844468732, 'precisions': [0.5033412970465012, 0.21597035040431267, 0.11024199462234173, 0.060397587670314945], 'brevity_penalty': 0.5445262852093052, 'length_ratio': 0.6219527952450685, 'translation_length': 28881, 'reference_length': 46436}


TrainOutput(global_step=10000, training_loss=2.316052233886719, metrics={'train_runtime': 2289.6833, 'train_samples_per_second': 109.185, 'train_steps_per_second': 4.367, 'total_flos': 7221661419110400.0, 'train_loss': 2.316052233886719, 'epoch': 5.0})

In [19]:
trainer.save_model("T5_overall")

In [20]:
text = "translate German to English: Hallo Leute"
from transformers import pipeline

translator = pipeline("translation_XX_to_YY", model="T5_overall")

translator(text)

/opt/conda/lib/python3.10/site-packages/transformers/generation/utils.py:1178: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


[{'translation_text': 'Hallo people'}]